In [1]:
import json
import time
import os
import glob
import os.path as osp
import numpy as np
import pandas as pd
import torch

os.chdir('/VOLUME/nia_vent_asynchrony')

In [2]:
import module.utils as cutils

In [5]:
org = 'snu'

In [6]:
label_path = {'snu': '/VOLUME/nia_vent_asynchrony/data/raw_data/baseline_220818/label/annotations_2022_NIA_MV_SNUH.json',
'aju': '/VOLUME/nia_vent_asynchrony/data/raw_data/baseline_220818/label/annotations_2022_NIA_MV_AUMC.json'}[org]
print(org, label_path)
# label_path = '/VOLUME/nia_vent_asynchrony/data/raw_data/baseline_220818/label/annotations_2022_NIA_MV_AUMC.json'
label = cutils.load_json(label_path)

snu /VOLUME/nia_vent_asynchrony/data/raw_data/baseline_220818/label/annotations_2022_NIA_MV_SNUH.json


In [7]:
len(label)

1069

In [7]:
label_index = 0
label[label_index].keys()

dict_keys(['id', 'annotations', 'drafts', 'predictions', 'data', 'meta', 'created_at', 'updated_at', 'inner_id', 'total_annotations', 'cancelled_annotations', 'total_predictions', 'project', 'updated_by'])

In [8]:
annotations = []
for label_i in label:
    file_path = label_i['data']['csv']
    for a_i in label_i['annotations']:
        if len(a_i['result'])>0:
            df = pd.DataFrame([i['value'] for i in a_i['result']])
            df.insert(0,'file_path', file_path)
            annotations.append(df)
df = pd.concat(annotations)
df['timeserieslabels'] = df['timeserieslabels'].str[0]
df_anno = df.drop(columns='instant')

In [10]:
ls = []
for label_i in label:
    file_path = label_i['data']['csv']
    for a_i in label_i['drafts']:
        if len(a_i['result'])>0:
            df = pd.DataFrame([i['value'] for i in a_i['result']])
            df.insert(0,'file_path', file_path)
            ls.append(df)
df = pd.concat(ls)
df['timeserieslabels'] = df['timeserieslabels'].str[0]
df_drafts = df.drop(columns='instant')

In [292]:
df_anno['timeserieslabels'].value_counts()

Premature cycling                       5021
Ineffective effort during expiration    4374
Delayed cycling                         1684
Double Triggering                       1067
Noise                                    272
Work shifting                             15
Auto PEEP                                  3
Name: timeserieslabels, dtype: int64

In [293]:
df_drafts['timeserieslabels'].value_counts()

Premature cycling                       370
Delayed cycling                          58
Ineffective effort during expiration     45
Double Triggering                         8
Name: timeserieslabels, dtype: int64

In [294]:
df_anno['timeserieslabels'].value_counts()

Premature cycling                       5021
Ineffective effort during expiration    4374
Delayed cycling                         1684
Double Triggering                       1067
Noise                                    272
Work shifting                             15
Auto PEEP                                  3
Name: timeserieslabels, dtype: int64

In [295]:
df_anno.tail()['file_path'].tolist()

['/data/local-files/?d=SNUH/1-002-002/1-002-002_003.csv',
 '/data/local-files/?d=SNUH/1-002-002/1-002-002_003.csv',
 '/data/local-files/?d=SNUH/1-002-002/1-002-002_003.csv',
 '/data/local-files/?d=SNUH/1-002-002/1-002-002_001.csv',
 '/data/local-files/?d=SNUH/1-002-002/1-002-002_001.csv']

In [296]:
df_anno.sort_values(['file_path','start'])

,file_path,start,end,timeserieslabels
0,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 02:54:17.500000+0900,2021-09-24 02:54:27.984000+0900,Noise
1,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 02:59:53.274000+0900,2021-09-24 02:59:54.524000+0900,Noise
0,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:01:05.629000+0900,2021-09-24 03:01:11.663000+0900,Ineffective effort during expiration
1,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:01:27.980000+0900,2021-09-24 03:01:30.513000+0900,Ineffective effort during expiration
3,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:07:21.466000+0900,2021-09-24 03:07:23.083000+0900,Premature cycling
...,...,...,...,...
0,/data/upload/2/2d6fd3e8-MICU_01_220312_110000....,2022-03-12 11:52:36.024000+0900,2022-03-12 11:52:46.808000+0900,Double Triggering
5,/data/upload/2/2d6fd3e8-MICU_01_220312_110000....,2022-03-12 11:52:47.008000+0900,2022-03-12 11:53:40.009000+0900,Noise
1,/data/upload/2/2d6fd3e8-MICU_01_220312_110000....,2022-03-12 11:53:40.259000+0900,2022-03-12 11:53:45.092000+0900,Double Triggering
6,/data/upload/2/2d6fd3e8-MICU_01_220312_110000....,2022-03-12 11:55:46.711000+0900,2022-03-12 11:55:56.478000+0900,Double Triggering


In [297]:
df_drafts['timeserieslabels'].value_counts()

Premature cycling                       370
Delayed cycling                          58
Ineffective effort during expiration     45
Double Triggering                         8
Name: timeserieslabels, dtype: int64

In [298]:
df_drafts.sort_values(['file_path','start'])

,file_path,start,end,timeserieslabels
1,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:07:21.466000+0900,2021-09-24 03:07:23.083000+0900,Premature cycling
2,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:07:47.600000+0900,2021-09-24 03:07:49.417000+0900,Premature cycling
0,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:07:57.800000+0900,2021-09-24 03:07:59.767000+0900,Premature cycling
3,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:08:23.934000+0900,2021-09-24 03:08:25.884000+0900,Premature cycling
4,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:08:29.818000+0900,2021-09-24 03:08:31.734000+0900,Premature cycling
...,...,...,...,...
1,/data/upload/2/2d6fd3e8-MICU_01_220312_110000....,2022-03-12 11:26:16.659000+0900,2022-03-12 11:26:20.693000+0900,Double Triggering
0,/data/upload/2/2d6fd3e8-MICU_01_220312_110000....,2022-03-12 11:26:32.926000+0900,2022-03-12 11:26:44.793000+0900,Double Triggering
1,/data/upload/2/2d6fd3e8-MICU_01_220312_110000....,2022-03-12 11:26:47.060000+0900,2022-03-12 11:26:54.477000+0900,Double Triggering
2,/data/upload/2/2d6fd3e8-MICU_01_220312_110000....,2022-03-12 11:37:03.622000+0900,2022-03-12 11:37:06.172000+0900,Double Triggering


In [299]:
# 84행 겹침
pd.merge(df_anno, df_drafts)

,file_path,start,end,timeserieslabels
0,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-28 00:00:26.722000+0900,2021-09-28 00:00:30.322000+0900,Delayed cycling
1,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-28 00:00:33.755000+0900,2021-09-28 00:00:36.938000+0900,Delayed cycling
2,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-28 00:00:44.722000+0900,2021-09-28 00:00:48.422000+0900,Delayed cycling
3,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-28 00:00:52.389000+0900,2021-09-28 00:00:55.522000+0900,Delayed cycling
4,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-28 00:01:09.922000+0900,2021-09-28 00:01:12.939000+0900,Delayed cycling
...,...,...,...,...
79,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:07:21.466000+0900,2021-09-24 03:07:23.083000+0900,Premature cycling
80,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:07:47.600000+0900,2021-09-24 03:07:49.417000+0900,Premature cycling
81,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:08:23.934000+0900,2021-09-24 03:08:25.884000+0900,Premature cycling
82,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:08:29.818000+0900,2021-09-24 03:08:31.734000+0900,Premature cycling


In [91]:
ann_df = df_anno.drop_duplicates()
# df = df_drafts.drop_duplicates()

In [92]:
# ann_df = pd.concat([df_anno, df_drafts]).drop_duplicates()

In [93]:
ann_df[ann_df['end']=='0NaN-NaN-NaN NaN:NaN:NaN.NaN000+00NaN']

,file_path,start,end,timeserieslabels
0,/data/local-files/?d=SNUH/1-004-004/1-004-004_...,2021-09-17 20:00:42.889000+0900,0NaN-NaN-NaN NaN:NaN:NaN.NaN000+00NaN,Noise
0,/data/local-files/?d=SNUH/1-004-004/1-004-004_...,2021-09-17 05:02:05.980000+0900,0NaN-NaN-NaN NaN:NaN:NaN.NaN000+00NaN,Ineffective effort during expiration
1,/data/local-files/?d=SNUH/1-004-004/1-004-004_...,2021-09-17 05:02:56.865000+0900,0NaN-NaN-NaN NaN:NaN:NaN.NaN000+00NaN,Ineffective effort during expiration
5,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-09-05 05:12:19.707000+0900,0NaN-NaN-NaN NaN:NaN:NaN.NaN000+00NaN,Double Triggering
6,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-09-05 05:37:30.987000+0900,0NaN-NaN-NaN NaN:NaN:NaN.NaN000+00NaN,Double Triggering
7,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-09-05 05:14:37.709000+0900,0NaN-NaN-NaN NaN:NaN:NaN.NaN000+00NaN,Double Triggering
8,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-09-05 05:14:47.810000+0900,0NaN-NaN-NaN NaN:NaN:NaN.NaN000+00NaN,Double Triggering
9,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-09-05 05:16:01.861000+0900,0NaN-NaN-NaN NaN:NaN:NaN.NaN000+00NaN,Double Triggering
10,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-09-05 05:13:27.025000+0900,0NaN-NaN-NaN NaN:NaN:NaN.NaN000+00NaN,Double Triggering
1,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-09-05 03:58:22.865000+0900,0NaN-NaN-NaN NaN:NaN:NaN.NaN000+00NaN,Noise


In [94]:
# df[df['end']=='0NaN-NaN-NaN NaN:NaN:NaN.NaN000+00NaN'].to_csv('/VOLUME/nia_vent_asynchrony/result/baseline_220818/label/snu_json_endtime_nan.csv', index=False)

In [95]:
ann_df = ann_df[ann_df['end']!='0NaN-NaN-NaN NaN:NaN:NaN.NaN000+00NaN']
# drop /data/upload/2/2d6fd3e8-MICU_01_*
ann_df = ann_df[ann_df['file_path'].str.contains('/data/local-files/', regex=False)]
ann_df['start'] = pd.to_datetime(ann_df['start'])
ann_df['end'] = pd.to_datetime(ann_df['end'])
ann_df['duration'] = ann_df['end']-ann_df['start']

# snu (path: '/data/local-files/?d=SNUH/1-005-005/1-005-005_117.csv')
ann_df['hospital_id_patient_id'] = ann_df['file_path'].str.split('/').str[-2].str.split('-').str[:2].str.join('-')
ann_df['wav_number'] = ann_df['file_path'].str.split('/').str[-1].str.split('_').str[-1].str.replace('.csv','').astype(int)
# aju (path: '/data/local-files/?d=AUMC/7f1601175da2753c0a66/201904261300_201904261400.csv')


/tmp/ipykernel_30207/2306700832.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df['wav_number'] = df['file_path'].str.split('/').str[-1].str.split('_').str[-1].str.replace('.csv','').astype(int)


In [51]:
ann_df['timeserieslabels'].value_counts()

Premature cycling                       5020
Ineffective effort during expiration    4367
Delayed cycling                         1684
Double Triggering                       1055
Noise                                    266
Work shifting                             15
Auto PEEP                                  3
Name: timeserieslabels, dtype: int64

In [53]:
ann_df['timeserieslabels'].isna().sum()

0

In [96]:
ann_df['label_annotation'] = 1 # asynchrony (true)
ann_df.loc[(ann_df['end']-ann_df['start'])<pd.Timedelta(minutes=1),'label_annotation'] = 2 # asynchrony (false)

# 0) normal 1) asynchrony (true) 2) asynchrony (false) 3) noise
# 만약 1분기준으로 à 1분이내 (false) ; 1분이상 (true) : 로 같이 구분되면 좋습니다.

ann_df.loc[ann_df['timeserieslabels']=='Noise','label_annotation'] = 3


In [97]:
ann_df.sort_values('duration').head(20)

,file_path,start,end,timeserieslabels,duration,hospital_id_patient_id,wav_number,label_annotation
0,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-26 13:36:28.383000+09:00,2021-09-26 13:36:28.383000+09:00,Premature cycling,0 days 00:00:00,1-002,432,2
18,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-25 05:44:58.902000+09:00,2021-09-25 05:44:58.902000+09:00,Premature cycling,0 days 00:00:00,1-002,222,2
16,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-25 04:14:20.733000+09:00,2021-09-25 04:14:20.733000+09:00,Premature cycling,0 days 00:00:00,1-002,207,2
8,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-25 04:25:12.271000+09:00,2021-09-25 04:25:12.271000+09:00,Premature cycling,0 days 00:00:00,1-002,209,2
20,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-25 09:59:11.625000+09:00,2021-09-25 09:59:11.642000+09:00,Ineffective effort during expiration,0 days 00:00:00.017000,1-002,264,2
5,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-25 06:31:22.950000+09:00,2021-09-25 06:31:22.983000+09:00,Premature cycling,0 days 00:00:00.033000,1-002,230,2
15,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-25 04:26:16.839000+09:00,2021-09-25 04:26:16.889000+09:00,Premature cycling,0 days 00:00:00.050000,1-002,209,2
20,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-26 12:29:09.337000+09:00,2021-09-26 12:29:09.404000+09:00,Premature cycling,0 days 00:00:00.067000,1-002,430,2
6,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-25 10:38:11.616000+09:00,2021-09-25 10:38:11.683000+09:00,Delayed cycling,0 days 00:00:00.067000,1-002,271,2
214,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-08-29 17:55:52.660000+09:00,2021-08-29 17:55:52.727000+09:00,Premature cycling,0 days 00:00:00.067000,1-003,23,2


In [98]:
(ann_df['end']-ann_df['start']).describe()

count                        12410
mean     0 days 00:00:59.134505721
std      0 days 00:06:17.258641670
min                0 days 00:00:00
25%         0 days 00:00:02.567000
50%         0 days 00:00:03.033000
75%         0 days 00:00:04.817000
max         0 days 00:59:29.571000
dtype: object

In [100]:
ann_df.groupby(['hospital_id_patient_id'])['wav_number'].max()

hospital_id_patient_id
1-002    588
1-003    390
1-004     80
1-005    141
Name: wav_number, dtype: int64

In [20]:
wav_dcm_path = {'snu':'/VOLUME/nia_vent_asynchrony/data/raw_data/baseline_220818/waveform_dcm/22_AIHub_26-2(Cons)_dcm(pdf)_서울대_1-002_1-021_ventilator_wav_file.csv', 
'aju': '/VOLUME/nia_vent_asynchrony/data/raw_data/baseline_220818/waveform_dcm/22_AIHub_26-2(Cons)_dcm(pdf)_아주대_2-001_010_ventilator.xlsx'}[org]
print(wav_dcm_path)
if wav_dcm_path.split('.')[-1]=='csv':
    wav_dcm = pd.read_csv(wav_dcm_path, encoding='cp949')
if wav_dcm_path.split('.')[-1]=='xlsx':
    wav_dcm = pd.read_excel(wav_dcm_path)
    wav_dcm = wav_dcm[wav_dcm['hospital_id']==2]
wav_dcm['hospital_id_patient_id'] = wav_dcm['hospital_id'].astype(str)+'-'+(wav_dcm['patient_id'] if wav_dcm['patient_id'].dtype=='O' else wav_dcm['patient_id'].apply(lambda x : '{:03d}'.format(x)))

/VOLUME/nia_vent_asynchrony/data/raw_data/baseline_220818/waveform_dcm/22_AIHub_26-2(Cons)_dcm(pdf)_서울대_1-002_1-021_ventilator_wav_file.csv


In [21]:
wav_dcm['hospital_id_patient_id'].unique()

array(['1-002', '1-003', '1-004', '1-005', '1-006', '1-007', '1-008',
       '1-009', '1-010', '1-011', '1-012', '1-013', '1-014', '1-015',
       '1-016', '1-017', '1-018', '1-019', '1-020', '1-021'], dtype=object)

In [22]:
# annotation에 더 많은 wav_number가 있으면 자름 (임시)
if org=='snu':
    max_wav_df = pd.DataFrame({'hospital_id_patient_id':['1-002','1-003','1-004'], 'max_wav': [105, 378, 80]})
# elif org=='aju':
#     max_wav_df = pd.DataFrame({'hospital_id_patient_id':['2-002', '2-003', '2-004', '2-005', '2-006', '2-007', '2-008','2-009', '2-010'], 'max_wav': [105, 378, 80]})
label_ann = pd.merge(df, max_wav_df)
label_ann = label_ann[label_ann.wav_number<=label_ann.max_wav]

In [23]:
max_wav_df['max_wav'].sum()*60*60/8

253350.0

In [24]:
max_wav_df['max_wav']

0    105
1    378
2     80
Name: max_wav, dtype: int64

In [25]:
max_wav_df['max_wav']

0    105
1    378
2     80
Name: max_wav, dtype: int64

In [27]:
max_wav_df

,hospital_id_patient_id,max_wav
0,1-002,105
1,1-003,378
2,1-004,80


In [28]:
label_ann

,file_path,start,end,timeserieslabels,hospital_id_patient_id,wav_number,duration,max_wav
0,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-08-30 05:58:46.870000+09:00,2021-08-30 05:58:52.170000+09:00,Premature cycling,1-003,35,0 days 00:00:05.300000,378
1,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-08-30 05:58:59.653000+09:00,2021-08-30 05:59:02.403000+09:00,Premature cycling,1-003,35,0 days 00:00:02.750000,378
2,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-08-30 05:59:04.737000+09:00,2021-08-30 05:59:07.070000+09:00,Premature cycling,1-003,35,0 days 00:00:02.333000,378
3,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-08-30 05:59:09.187000+09:00,2021-08-30 05:59:12.270000+09:00,Premature cycling,1-003,35,0 days 00:00:03.083000,378
4,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-08-30 05:59:14.870000+09:00,2021-08-30 05:59:17.137000+09:00,Premature cycling,1-003,35,0 days 00:00:02.267000,378
...,...,...,...,...,...,...,...,...
12255,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:08:23.934000+09:00,2021-09-24 03:08:25.884000+09:00,Premature cycling,1-002,3,0 days 00:00:01.950000,105
12256,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:08:29.818000+09:00,2021-09-24 03:08:31.734000+09:00,Premature cycling,1-002,3,0 days 00:00:01.916000,105
12257,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:10:45.704000+09:00,2021-09-24 03:10:48.470000+09:00,Double Triggering,1-002,3,0 days 00:00:02.766000,105
12258,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 02:54:17.500000+09:00,2021-09-24 02:54:27.984000+09:00,Noise,1-002,1,0 days 00:00:10.484000,105


In [29]:
ann_df.groupby('hospital_id_patient_id')['wav_number'].unique().apply(len)

hospital_id_patient_id
1-002    479
1-003    311
1-004     80
1-005     10
Name: wav_number, dtype: int64

In [30]:
ann_df

,file_path,start,end,timeserieslabels,hospital_id_patient_id,wav_number,duration
0,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-08-30 05:58:46.870000+09:00,2021-08-30 05:58:52.170000+09:00,Premature cycling,1-003,35,0 days 00:00:05.300000
1,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-08-30 05:58:59.653000+09:00,2021-08-30 05:59:02.403000+09:00,Premature cycling,1-003,35,0 days 00:00:02.750000
2,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-08-30 05:59:04.737000+09:00,2021-08-30 05:59:07.070000+09:00,Premature cycling,1-003,35,0 days 00:00:02.333000
3,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-08-30 05:59:09.187000+09:00,2021-08-30 05:59:12.270000+09:00,Premature cycling,1-003,35,0 days 00:00:03.083000
4,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-08-30 05:59:14.870000+09:00,2021-08-30 05:59:17.137000+09:00,Premature cycling,1-003,35,0 days 00:00:02.267000
...,...,...,...,...,...,...,...
5,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:08:23.934000+09:00,2021-09-24 03:08:25.884000+09:00,Premature cycling,1-002,3,0 days 00:00:01.950000
6,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:08:29.818000+09:00,2021-09-24 03:08:31.734000+09:00,Premature cycling,1-002,3,0 days 00:00:01.916000
7,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:10:45.704000+09:00,2021-09-24 03:10:48.470000+09:00,Double Triggering,1-002,3,0 days 00:00:02.766000
0,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 02:54:17.500000+09:00,2021-09-24 02:54:27.984000+09:00,Noise,1-002,1,0 days 00:00:10.484000


In [31]:
tmp = pd.merge(wav_dcm, ann_df, how='right')
tmp[tmp['wav_file_path'].isna()]['file_path'].nunique()
tmp[tmp['wav_file_path'].isna()]['file_path'].unique()

array(['/data/local-files/?d=SNUH/1-003-003/1-003-003_267.csv',
       '/data/local-files/?d=SNUH/1-003-003/1-003-003_266.csv',
       '/data/local-files/?d=SNUH/1-003-003/1-003-003_265.csv',
       '/data/local-files/?d=SNUH/1-003-003/1-003-003_264.csv',
       '/data/local-files/?d=SNUH/1-003-003/1-003-003_263.csv',
       '/data/local-files/?d=SNUH/1-003-003/1-003-003_262.csv',
       '/data/local-files/?d=SNUH/1-003-003/1-003-003_261.csv',
       '/data/local-files/?d=SNUH/1-003-003/1-003-003_260.csv',
       '/data/local-files/?d=SNUH/1-003-003/1-003-003_259.csv',
       '/data/local-files/?d=SNUH/1-003-003/1-003-003_257.csv',
       '/data/local-files/?d=SNUH/1-003-003/1-003-003_256.csv',
       '/data/local-files/?d=SNUH/1-003-003/1-003-003_255.csv',
       '/data/local-files/?d=SNUH/1-003-003/1-003-003_254.csv',
       '/data/local-files/?d=SNUH/1-003-003/1-003-003_253.csv',
       '/data/local-files/?d=SNUH/1-003-003/1-003-003_252.csv',
       '/data/local-files/?d=SNUH/1-003-

waveform data 불러오기

In [33]:
wavecsv_list = glob.glob('/VOLUME/nia_vent_asynchrony/data/raw_data/baseline_220818/waveform/snu/csv/*/*flow/*.csv.gz')
dest_dir = '/VOLUME/nia_vent_asynchrony/data/processed_data'

In [34]:
len(wavecsv_list)

563

In [ ]:


df_list = []
for i, wave_i in enumerate(wavecsv_list): # # '/VOLUME/nia_vent_asynchrony/data/raw_data/baseline_220818/waveform/snu/csv/1-002/1-002-c-flow/1-002-c-flow_004.csv.gz'
    wav = pd.read_csv(wave_i, parse_dates=['Time'])
    wav['Intellivue/AWP_WAV'] = pd.read_csv(wave_i.replace('flow','pre'), parse_dates=['Time'])['Intellivue/AWP_WAV']
    n_instance = int((wav['Time'].max()-wav['Time'].min()).total_seconds()/instance_length_sec)
    print(i, n_instance)

    wav_starttime = wav['Time'].min()
    # tdi = 3
    for i in range(n_instance):
        one_instance_wav = wav[(wav['Time']>=wav_starttime+tdi*instance_length_sec_td)&(wav['Time']<wav_starttime+((tdi+1)*instance_length_sec_td))]
        one_instance = pd.Series({'flow_path': wave_i, 
            'starttime': one_instance_wav.min()['Time'], 'endtime': one_instance_wav.max()['Time'], 
            'data': one_instance_wav[['Intellivue/FLOW_WAV','Intellivue/AWP_WAV']].values})
        df_list.append(one_instance)

In [45]:
instance_length_sec = 8
instance_length_sec_td = pd.Timedelta(seconds=instance_length_sec)

def get_wave_instance_per_waveformfile(i):
    if i%100==0: print(i)
    
    wave_i = wavecsv_list[i]
    wav = pd.read_csv(wave_i, parse_dates=['Time'])
    wav['Intellivue/AWP_WAV'] = pd.read_csv(wave_i.replace('flow','pre'), parse_dates=['Time'])['Intellivue/AWP_WAV']
    n_instance = int((wav['Time'].max()-wav['Time'].min()).total_seconds()/instance_length_sec)
    # print(i, n_instance)

    wav_starttime = wav['Time'].min()
    df_list = []
    for tdi in range(n_instance):
        one_instance_wav = wav[(wav['Time']>=wav_starttime+tdi*instance_length_sec_td)&(wav['Time']<wav_starttime+((tdi+1)*instance_length_sec_td))]
        one_instance = pd.Series({'flow_path': wave_i, 
            'starttime': one_instance_wav.min()['Time'], 'endtime': one_instance_wav.max()['Time'], 
            'data': one_instance_wav[['Intellivue/FLOW_WAV','Intellivue/AWP_WAV']].values
            })
        df_list.append(one_instance)
    return pd.concat(df_list, axis=1).transpose()

In [46]:
import multiprocessing
since = time.time()
n_cpu = multiprocessing.cpu_count()
n_threads = 30
print(f'no. cpu existed : {n_cpu}, use {n_threads} threads')

print(len(wavecsv_list))
pool = multiprocessing.Pool(processes=n_threads)
result = pool.map(get_wave_instance_per_waveformfile, range(len(wavecsv_list)))
# result = pool.map(get_wave_instance_per_waveformfile, range(6))
pool.close()
pool.join()

print('aggregate all instances')
data = []
for i in result :
    if len(i)>0:
        data.append(i)
datadf = pd.concat(data)
print(since-time.time())

no. cpu existed : 104, use 30 threads
563
0
100
200
300
400
500
aggregate all instances
-214.64339923858643


In [47]:
datadf

,flow_path,starttime,endtime,data
0,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-28 02:00:21.831158+09:00,2021-09-28 02:00:29.814651+09:00,"[[40.69999694824219, -0.1450004577636718], [40..."
1,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-28 02:00:29.831318+09:00,2021-09-28 02:00:37.814811+09:00,"[[42.600006103515625, 8.299999237060547], [39...."
2,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-28 02:00:37.831478+09:00,2021-09-28 02:00:45.814971+09:00,"[[89.39999389648438, 4.47499942779541], [92.30..."
3,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-28 02:00:45.831638+09:00,2021-09-28 02:00:53.815131+09:00,"[[34.80000305175781, -0.0399999618530273], [35..."
4,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-28 02:00:53.831798+09:00,2021-09-28 02:01:01.815291+09:00,"[[40.600006103515625, -0.2350006103515625], [4..."
...,...,...,...,...
437,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-16 01:58:56.297212+09:00,2021-09-16 01:59:04.280705+09:00,"[[27.199996948242188, 5.420000076293945], [27...."
438,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-16 01:59:04.297372+09:00,2021-09-16 01:59:12.280865+09:00,"[[32.0, 4.835000038146973], [32.5, 4.444999694..."
439,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-16 01:59:12.297532+09:00,2021-09-16 01:59:20.281025+09:00,"[[44.0, 10.369998931884766], [43.5, 10.3699989..."
440,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-16 01:59:20.297692+09:00,2021-09-16 01:59:28.281185+09:00,"[[20.600006103515625, 5.494999885559082], [21...."


In [48]:
dest_path= osp.join(dest_dir, org, 'instance.pkl')
print(dest_path)
datadf.to_pickle(dest_path)

/VOLUME/nia_vent_asynchrony/data/processed_data/snu/instance.pkl


In [49]:
!du -sh /VOLUME/nia_vent_asynchrony/data/processed_data/snu/instance.pkl

1.8G	/VOLUME/nia_vent_asynchrony/data/processed_data/snu/instance.pkl


In [101]:
tmp = datadf['flow_path'].str.split('/').str[-1].str.split('_').apply(pd.Series)
tmp.columns = ['hospital_id_patient_id', 'wav_number']
tmp['hospital_id_patient_id'] = tmp['hospital_id_patient_id'].str.replace('-c-flow','')
tmp['wav_number'] = tmp['wav_number'].str.replace('.csv.gz','').astype(int)
datadf_info = pd.concat([datadf, tmp], axis=1)

/tmp/ipykernel_30207/2608231346.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  tmp['wav_number'] = tmp['wav_number'].str.replace('.csv.gz','').astype(int)


In [71]:
tmp

,hospital_id_patient_id,wav_number
0,1-002,91
1,1-002,91
2,1-002,91
3,1-002,91
4,1-002,91


In [62]:
ann_df['file_path'].iloc[0], datadf['flow_path'].iloc[0]

('/data/local-files/?d=SNUH/1-003-003/1-003-003_035.csv',
 '/VOLUME/nia_vent_asynchrony/data/raw_data/baseline_220818/waveform/snu/csv/1-002/1-002-c-flow/1-002-c-flow_091.csv.gz')

In [74]:
datadf_info

,flow_path,starttime,endtime,data,hospital_id_patient_id,wav_number
0,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-28 02:00:21.831158+09:00,2021-09-28 02:00:29.814651+09:00,"[[40.69999694824219, -0.1450004577636718], [40...",1-002,91
1,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-28 02:00:29.831318+09:00,2021-09-28 02:00:37.814811+09:00,"[[42.600006103515625, 8.299999237060547], [39....",1-002,91
2,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-28 02:00:37.831478+09:00,2021-09-28 02:00:45.814971+09:00,"[[89.39999389648438, 4.47499942779541], [92.30...",1-002,91
3,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-28 02:00:45.831638+09:00,2021-09-28 02:00:53.815131+09:00,"[[34.80000305175781, -0.0399999618530273], [35...",1-002,91
4,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-28 02:00:53.831798+09:00,2021-09-28 02:01:01.815291+09:00,"[[40.600006103515625, -0.2350006103515625], [4...",1-002,91
...,...,...,...,...,...,...
437,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-16 01:58:56.297212+09:00,2021-09-16 01:59:04.280705+09:00,"[[27.199996948242188, 5.420000076293945], [27....",1-004,40
438,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-16 01:59:04.297372+09:00,2021-09-16 01:59:12.280865+09:00,"[[32.0, 4.835000038146973], [32.5, 4.444999694...",1-004,40
439,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-16 01:59:12.297532+09:00,2021-09-16 01:59:20.281025+09:00,"[[44.0, 10.369998931884766], [43.5, 10.3699989...",1-004,40
440,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-16 01:59:20.297692+09:00,2021-09-16 01:59:28.281185+09:00,"[[20.600006103515625, 5.494999885559082], [21....",1-004,40


In [130]:
ann_df

,file_path,start,end,timeserieslabels,duration,hospital_id_patient_id,wav_number,label_annotation
0,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-08-30 05:58:46.870000+09:00,2021-08-30 05:58:52.170000+09:00,Premature cycling,0 days 00:00:05.300000,1-003,35,2
1,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-08-30 05:58:59.653000+09:00,2021-08-30 05:59:02.403000+09:00,Premature cycling,0 days 00:00:02.750000,1-003,35,2
2,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-08-30 05:59:04.737000+09:00,2021-08-30 05:59:07.070000+09:00,Premature cycling,0 days 00:00:02.333000,1-003,35,2
3,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-08-30 05:59:09.187000+09:00,2021-08-30 05:59:12.270000+09:00,Premature cycling,0 days 00:00:03.083000,1-003,35,2
4,/data/local-files/?d=SNUH/1-003-003/1-003-003_...,2021-08-30 05:59:14.870000+09:00,2021-08-30 05:59:17.137000+09:00,Premature cycling,0 days 00:00:02.267000,1-003,35,2
...,...,...,...,...,...,...,...,...
5,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:08:23.934000+09:00,2021-09-24 03:08:25.884000+09:00,Premature cycling,0 days 00:00:01.950000,1-002,3,2
6,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:08:29.818000+09:00,2021-09-24 03:08:31.734000+09:00,Premature cycling,0 days 00:00:01.916000,1-002,3,2
7,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 03:10:45.704000+09:00,2021-09-24 03:10:48.470000+09:00,Double Triggering,0 days 00:00:02.766000,1-002,3,2
0,/data/local-files/?d=SNUH/1-002-002/1-002-002_...,2021-09-24 02:54:17.500000+09:00,2021-09-24 02:54:27.984000+09:00,Noise,0 days 00:00:10.484000,1-002,1,3


In [119]:
datadf_info['instance_index'] = range(len(datadf_info))

In [136]:
def annotate_each_label(label_num:int):
    ann_data = pd.merge(datadf_info, ann_df[ann_df['label_annotation']==label_num], on=['hospital_id_patient_id','wav_number'])
    cond1 = (ann_data['starttime']<=ann_data['start'])&(ann_data['endtime']>=ann_data['end']) # anntation이 instance 에 포함
    cond2 = (ann_data['starttime']<=ann_data['start'])&(ann_data['endtime']>=ann_data['start']) # annotation start가 instance 에 포함
    cond3 = (ann_data['starttime']<=ann_data['end'])&(ann_data['endtime']>=ann_data['end']) # annotation end가 instance 에 포함
    ann_index = ann_data[cond1|cond2|cond3]['instance_index']
    return ann_index

In [138]:
ann_index_3 = annotate_each_label(3)
ann_index_2 = annotate_each_label(2)
ann_index_1 = annotate_each_label(1)

In [139]:
print(len(ann_index_1), len(ann_index_2), len(ann_index_3))

(67, 395, 26)

In [141]:
datadf_info['label'] = 0
datadf_info.loc[datadf_info['instance_index'].isin(ann_index_3),'label'] = 3
datadf_info.loc[datadf_info['instance_index'].isin(ann_index_2),'label'] = 2
datadf_info.loc[datadf_info['instance_index'].isin(ann_index_1),'label'] = 1

In [143]:
datadf_info.label.value_counts()

0    242157
2       373
1        64
3        24
Name: label, dtype: int64

In [145]:
(datadf_info.groupby('hospital_id_patient_id')['endtime'].max()-datadf_info.groupby('hospital_id_patient_id')['starttime'].min()).dt.total_seconds().sum()/8

259680.995570375

In [128]:
4.287119e+05

428711.9

In [146]:
dest_path = osp.join(dest_dir, org, 'instance_with_label_snu.pkl')
print(dest_path)
datadf_info.to_pickle(dest_path)

/VOLUME/nia_vent_asynchrony/data/processed_data/snu/instance_with_label_snu.pkl


In [147]:
datadf_info

,flow_path,starttime,endtime,data,hospital_id_patient_id,wav_number,instance_index,label
0,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-28 02:00:21.831158+09:00,2021-09-28 02:00:29.814651+09:00,"[[40.69999694824219, -0.1450004577636718], [40...",1-002,91,0,0
1,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-28 02:00:29.831318+09:00,2021-09-28 02:00:37.814811+09:00,"[[42.600006103515625, 8.299999237060547], [39....",1-002,91,1,0
2,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-28 02:00:37.831478+09:00,2021-09-28 02:00:45.814971+09:00,"[[89.39999389648438, 4.47499942779541], [92.30...",1-002,91,2,0
3,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-28 02:00:45.831638+09:00,2021-09-28 02:00:53.815131+09:00,"[[34.80000305175781, -0.0399999618530273], [35...",1-002,91,3,0
4,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-28 02:00:53.831798+09:00,2021-09-28 02:01:01.815291+09:00,"[[40.600006103515625, -0.2350006103515625], [4...",1-002,91,4,0
...,...,...,...,...,...,...,...,...
437,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-16 01:58:56.297212+09:00,2021-09-16 01:59:04.280705+09:00,"[[27.199996948242188, 5.420000076293945], [27....",1-004,40,242613,0
438,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-16 01:59:04.297372+09:00,2021-09-16 01:59:12.280865+09:00,"[[32.0, 4.835000038146973], [32.5, 4.444999694...",1-004,40,242614,0
439,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-16 01:59:12.297532+09:00,2021-09-16 01:59:20.281025+09:00,"[[44.0, 10.369998931884766], [43.5, 10.3699989...",1-004,40,242615,0
440,/VOLUME/nia_vent_asynchrony/data/raw_data/base...,2021-09-16 01:59:20.297692+09:00,2021-09-16 01:59:28.281185+09:00,"[[20.600006103515625, 5.494999885559082], [21....",1-004,40,242616,0
